#  Cogmaster, Neuro 102: hands on (f)MRI
__Content creator:__ Florent Meyniel, NeuroSpin, CEA Paris-Saclay

I would like to acknowledge my colleagues [Le Ster _et al._ (2019)](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0225286) for sharing their data on the OpenNeuro platform [here](https://openneuro.org/datasets/ds002606), part of which I am going to use here.


---
# First steps

## How to use a python notebook?
If you are running this notebook in Google colab, first make sure that you are connected to a computer. If "Connected" or "RAM and disk" appears in the top, right-hand side corner, you are fine! otherwise, click on "Connect".

You can skip the remainder of this section if you already know about python notebook (jupyter-notebook, google colab).

The goal of today's session is *not* to learn how to program with Python. However, we will use Python to run some examples and do computations, so here is a quick introduction to Python notebook.

A notebook mixes text, lines of code that can be executed, and results that are displayed.

There are different ways to execute a cell with code. All are equivalent:
- put the cursor in the line of code and press "SHIFT"+"ENTER".
- or click on the "play" icon (the triangle) on the left-hand side of the cell.
For example, the next line instructs python to compute "1+1". Execute the line of code to display the (expected) result.

In [ ]:
1+1

The number in squared brackets indicates the order in which a "cell" was executed. If you execute again the cell above, you will see that the number within the squared brackets increases. 

Now, you are going to type your own piece of code in the empty line below. Let's try 2+1

If you want to insert your own new cell and enter your code there, click on the "+" button in the menu bar at the top of this page. Insert a new cell below and a new code, e.g. 2+2

## Set-up the environment

The following cell installs the libraries that we will use.

In [ ]:
# install libraries
! pip install -U nilearn
! pip install -U nistats

This cell downloads the data that we need.

In [ ]:
# clone the git repository to get the data
! git clone https://github.com/florentmeyniel/cogmaster_2020_neuro102.git

Wait until the download is complete. When you see the folder:
>cogmaster_2020_neuro102

in the directory (in the left panel), you can continue and execute the next cell to move in the folder containing the data.

In [ ]:
cd cogmaster_2020_neuro102/

Now, do various imports.

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm 

from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix
from nilearn.glm.first_level import run_glm
from nilearn.glm import compute_contrast
from nilearn.glm import fdr_threshold
from nilearn import plotting
from nilearn.input_data import NiftiMasker
from nilearn import surface
from nilearn import datasets

%matplotlib inline

# Part 1: Different subjects, different brains
The study by Le Ster and colleagues, as most neuro-imaging studies, is performed at the level of a group of subjects in order to estimate the statistical significance of the results. 

In a new tab, open the [link to the data stored on OpenNeuro](https://openneuro.org/datasets/ds002606).

**Q1-1: How many participants were included in the study?**

We are going to explore the data set. Browse the data: open sub_03 / ses-01 / anat and open the anatomical image of this subject by clicking on the "view" icon (open it in a new tab). Or alternatively, click [there](https://openneuro.org/datasets/ds002606/versions/1.0.1/file-display/sub-03:ses-01:anat:sub-03_ses-01_T1w.nii.gz) (again, open this link in a new tab).

**Q1-2: Knowing that this is an anatomical image, what kind of weighting was used? T1 or T2?**

**Q1-3: What do you see in the anatomical image?**

If the image looks too dark, you can adjust the grey levels by clicking on the black-to-white gradient icon in the bottom right-hand corner. I find that the range 0 to 200 provides a nice image. You can navigate in this image by moving the cursor.

**Q1-4: Can you see the nose, eyes, ears of the participants? Why?**

In another tab, display the anatomy of the [4th subject](https://openneuro.org/datasets/ds002606/versions/1.0.1/file-display/sub-04:ses-01:anat:sub-04_ses-01_T1w.nii.gz).

**Q1-5: Compare the two anatomies. Do they look similar?**

In order to compare quantitatively the anatomy of those two participants, we are going to measure the distance between two anatomical landmarks: the anterior and posterior commissures. Beyond this exercise, it is interesting to know about those structures, because the most widely used frames of reference for brain anatomy uses:
- the anterior commissure as its origin.
- the axis passing through the anterior and posterior commissures as the antero-posterior axis (labeled y). The plane that contains this axis and that is oriented so as to separate the left and right hemispheres is the sagital plane.
- the axis orthogonal to the sagital plane corresponds to the left-right axis (labeled x), it is embedded in the axial plane
- the axis orthogonoal to those two axes is the bottom-to-top axis (labeled z).
This frame of reference is used by the [Talairach](http://en.wikipedia.org/wiki/Jean_Talairach) and the Montreal Neurological Institute [(MNI)](http://www.bic.mni.mcgill.ca/~louis/stx_history.html) coordinate systems.

You are going to explore the anatomy of those two subjects. Locate the anterior and posterior commissures and mark down their coordinate. In order to locate those two landmarks, use the description and pictures provided by this [guide](https://neuroimage.usc.edu/brainstorm/CoordinateSystems) (see the section "Talairach coordinates").

To compute the distance between to two landmarks A and B, for each subject, we use the following formula:
$\sqrt{(A_x - B_x)^2 + (A_y - B_y)^2 + (A_z - B_z)^2 }$ which is implemented in the cell below. 

In [ ]:
def distance(A, B):
    return np.sqrt(np.sum([(a-b)**2 for a, b in zip(A, B)]))

# Subject #3 (replace AC and PC with the values that you have marked down)
AC = [0, 0, 0]
PC = [0, 0, 0]
print('Subject #3, AC to PC=', distance(AC, PC), ' mm')

# Subject #4 (replace AC and PC with the values that you have marked down)
AC = [0, 0, 0]
PC = [0, 0, 0]
print('Subject #4, AC to PC=', distance(AC, PC), ' mm')


**Q1-6: Which subject has the largest brain among those two participants?**

**Q1-7: What problems do you foresee for group-level analysis?**

In order to ensure a voxel-to-voxel correspondence across participants, we normalize the anatomy. The simplest way of normalizing anatomy is to move (translation, rotation) and stretch (along the three possible axes) the brain of each subject so as to best match the anatomy of another subject that serves as reference. 

In practice, the reference anatomy is not from a subject in our group, but is a reference anatomy used by everybody in the research community. Having a common reference anatomy across studies ensures that the brain coordinates are the same across studies. The Montreal Neurological Institute has created such a reference anatomy which is widely used nowadays. 

The normalization is estimated based on the anatomical image of the subject, and then applied to the functional images, such that the coordinate system of the functional images becomes normalized too.

If you have completed this section and need to wait before doing the next one, or if you want to learn more, you can visit:
- this online version of the [text-book](https://www.fil.ion.ucl.ac.uk/spm/doc/books/hbf2/), especially section 2-3
- those [slides](https://www.fil.ion.ucl.ac.uk/spm/course/slides19-may/), especially 01_fMRI_preprocessing.pptx

which are provided by the SPM team at University College London.

## Pre-processing of data

Normalization is one of the many steps of a standard pre-preprocessing pipeline.
In this TD, we are going to use data which have been pre-processed following those steps:
- slice timing correction: timeseries in each slice are corrected so as to have the same timing (i.e. as if they had been sampled at the same time). 
- realignment: consecutive volumes are realigned to correct for volume-to-volume movement of subjects
- distortion of corrections induced by local field inhomogeneity
- coregistration of the functional and anatomical images
- normalisation of the anatomical image to a standard template. The same deformation is applied to functional images.
- spatial smoothing of the data.
- the data is spatially resampled, from 1.6 mm to 4 mm isotropic.

**Q1-8: Comment each step: why do we do that?**

Note that I reduced the spatial resolution in order to speed up computations. This is just for educational purposes; in practice, higher resolution is an interesting feature that one wants to leverage!

---
# Part 2: Regression analysis and the General Linear Model (GLM)

## Example data set: A simple functional "localizer"

The data we are going to analyze were collected during a "localizer" paradigm, developped by [P. Pinel et al, BMC 2007](https://bmcneurosci.biomedcentral.com/articles/10.1186/1471-2202-8-91).


Here is (a portion of) the abstract of this paper:
>Although cognitive processes such as reading and calculation are associated with reproducible cerebral networks, inter-individual variability is considerable. Understanding the origins of this variability will require the elaboration of large multimodal databases compiling behavioral, anatomical, genetic and functional neuroimaging data over hundreds of subjects. With this goal in mind, we designed a simple and fast acquisition procedure based on a 5-minute functional magnetic resonance imaging (fMRI) sequence that can be run as easily and as systematically as an anatomical scan, and is therefore used in every subject undergoing fMRI in our laboratory. This protocol captures the cerebral bases of auditory and visual perception, motor actions, reading, language comprehension and mental calculation at an individual level.

Here is a figure of the paper that describes the task:
![pinel](https://media.springernature.com/full/springer-static/image/art%3A10.1186%2F1471-2202-8-91/MediaObjects/12868_2007_Article_389_Fig1_HTML.jpg)

## Building a design matrix
We will build a design matrix that corresponds to this task. The following file lists the events and their timing in the task.

Execute the code that loads the data and browse its content.  
**Q2-1: Given the label of trials, is this design amenable to categorical or parametric regressors?**

In [ ]:
# load event file
task_file = 'sub-01_ses-01_locAP-sms-1-6iso-events.tsv'
events = pd.read_table(task_file)
events

The code below will construct a design matrix for this task.  
Note that the ten first columns correspond to task-related regressors.  
**Q2-2: Explain how those regressors are constructed**  

The design matrix also has columns mvt0 to mvt5 corresponding to the volume-to-volume movements of subjects (3 parameters for translation along x, y, z axis, and 3 parameters for rotation pitch, roll, yaw), and drift parameters (drift_1 to drift_4).  
**Q2-3: Why do we include those extra regressors in the design matrix?**

In [ ]:
# load movement parameters
movement = pd.read_csv('rp_asub-01_locAP_multiband_1_6iso.txt', sep='\s+',
                       header=None, names=[f"mvt{k}" for k in range(6)])

# get frame times (when each frame, a.k.a. volume, is collected)
TR = 1.2285
frame_times = np.arange(movement.shape[0])*TR

# make design matrix
design_matrix = make_first_level_design_matrix(frame_times,
                                               events, 
                                               drift_model='cosine',
                                               high_pass=1/128,
                                               add_regs=movement,
                                               add_reg_names=[name for name in movement.columns])
plot_design_matrix(design_matrix)

We following line of code will zoom onto one column: the "audio_left_hand" column. The blue curve is the predicted fMRI timecourse elicited by this stimulus, and the dots denote the exact timing of those events.

In [ ]:
plt.plot(frame_times, design_matrix['audio_left_hand'], label='BOLD')
plt.plot(events.loc[events['trial_type']=='audio_left_hand', 'onset'].values, 
         [0]*len(events.loc[events['trial_type']=='audio_left_hand', 'onset'].values),
         'o', label='left click, audio instruction')
plt.legend()
plt.xlabel('Time (s)')
plt.ylabel('Signal')

**Q2-4: We often say that the fMRI response is indirect, delayed and slow. Why?**  
**Q2-5: The last peak is nearly twice as large as the other peaks. Why?**

## Analysis of one voxel
We are now going to analyze the activity of a single example voxel. First, load the data. 

In [ ]:
# load data
masker = NiftiMasker(mask_strategy='epi',
                    detrend=True,
                    high_pass=1/128,
                    t_r=TR)
fMRI_data = masker.fit_transform('swtrasub-01_locAP_multiband_4mm.nii')

Now extract an example voxel and plot its activity.  
**Q2-6: Describe the timeseries**

In [ ]:
# let's focus on one voxel
voxel_index = 4999
example_voxel = fMRI_data[:, voxel_index]

# and plot it's activity
plt.plot(frame_times, fMRI_data[:, voxel_index])
plt.xlabel('Time (s)')
plt.ylabel('BOLD signal')

The following cell will compute and plot the predicted fMRI signal triggered by three types of events:
- a click with the left hand
- a click with the right hand
- flashing a checkerboard

In [ ]:
# Plot prediction signal
predicted = {'left_click': design_matrix['audio_left_hand'].values + design_matrix['video_left_hand'].values,
             'right_click':design_matrix['audio_right_hand'].values + design_matrix['video_right_hand'].values,
              'checkerboard': design_matrix['horizontal_checkerboard'].values + \
             design_matrix['vertical_checkerboard'].values}

for k, stimulus in enumerate(predicted):
    plt.subplot(3,1,k+1)
    plt.plot(frame_times, predicted[stimulus])
    plt.xlabel('Time (s)')
    plt.ylabel('BOLD signal')
    plt.title(stimulus)
plt.tight_layout()

**Q2-7: Compare visually the observed timeseries and those three predicted timeseries. Which one is most similar to the observed timeseries?**  
The following cell plots the observed and predicted signal one against the other.  
**Q2-8: Interpret the X-Y plot below**

In [ ]:
# plot predicted vs. observed
for k, stimulus in enumerate(predicted):
    plt.subplot(1,3,k+1)
    plt.plot(predicted[stimulus], example_voxel, '.')
    plt.xlabel('Predicted')
    plt.ylabel('Observed')
    plt.title(stimulus)
plt.tight_layout()

**Q2-9 What is a regression weight (a.k.a. "beta")?**  
The cell below computes and plots the (ordinary least square) estimate of the regression weights.  
Note that I use the formula below. In practice, this is packaged in your analysis software (as we will see later).

In [ ]:
# Normalize regressors
X = design_matrix.values
Xz = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
Xz[:,-1] = 1

# Compute the regression weights
beta = np.linalg.inv(Xz.T @ Xz) @ Xz.T @ example_voxel
plt.bar(np.arange(len(beta)), beta)
plt.xticks(ticks=np.arange(len(beta)),
          labels=[name for name in design_matrix.columns],
          rotation=60, ha='right')
plt.ylabel('beta')

The regressors have been normalized so that the magnitude of the betas can be compared among each other.  
**Q2-10: Interpret the betas. What appears to be a likely cause of the activity in this voxel?**  

In the course, we saw that we can test effects in the data with contrasts. Below I specify two example contrasts:
- "left click elicits more signal than right click" (displayed)
- "audio stimuli elicits more signal than visual stimuli" (not displayed below).  

**Q2-11: How is a contrast constructed?**

In [ ]:
# Specify contrast
contrasts = {
    'left_hand': np.array([1 if 'left_hand' in name else 0 for name in design_matrix.columns]),
    'right_hand': np.array([1 if 'right_hand' in name else 0 for name in design_matrix.columns]),
    'audio': np.array([1 if 'audio' in name else 0 for name in design_matrix.columns]),
    'video': np.array([1 if 'video' in name else 0 for name in design_matrix.columns]),}
contrasts['left - right click'] = contrasts['left_hand'] - contrasts['right_hand']
contrasts['audio - video'] = contrasts['audio'] - contrasts['video']

# Show contrast
print(contrasts['left - right click'])
plt.bar(np.arange(len(contrasts['left - right click'])),
        contrasts['left - right click'])
plt.xticks(ticks=np.arange(len(contrasts['left - right click'])),
          labels=[name for name in design_matrix.columns],
          rotation=60, ha='right')
plt.title('Contrast: '+ 'left - right click')

Given the name of the columns in the design matrix, **Q2-12: What does the following contrast correspond to?**
[1 0 0 -1 0 0 1 0 0 -1 0 0 0 0 0 0 0 0 0 0 0]  

Below I compute the t-value corresponding to the two contrasts "left - right click" and "audio - video".  
**Q2-13: Which t-value is the largest? Interpret**  

Note that I use the formula for educational purpose. This computation is also packaged in standard analysis softwares. 

In [ ]:
# compute the t-value of two contrast
def compute_tvalue(y, X, beta, c):
    c = c[:, np.newaxis]
    res = y - X @ beta
    tval = c.T @ beta / (np.std(res) * np.sqrt(c.T @ np.linalg.inv(X.T @ X) @ c))
    return tval[0][0]

for contrast_id in ['left - right click', 'audio - video']:
    print(contrast_id, ', tvalue=',
          compute_tvalue(example_voxel, Xz, beta,
                         contrasts[contrast_id]))

If time permits, you can rerun the cells above, starting from "# let's focus on one voxel" where you change the index of the voxel (the original value is 4999, you can pick any value from 0 to 26942, which is the number of voxels in this data set). Execute the cells in the order in which they appear.

In pratice, one does not compute the beta weights and tvalues by hand like I did here, but instead uses existing packages.
For instance here is a [regression function](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) available in scikit-learn that return the beta estimates.
And [here](https://www.statsmodels.org/stable/regression.html) is another regression function that returns the beta estimates and their statistics (t-value, p-value, etc.), in statsmodels.

---
# Part 3: Whole brain analysis of the "localizer" with encoding models  

## Estimation and contrast: Testing effects

We are now going to analyze all voxels, not just an example.  
**Q3-1: How does one call this type of regression which is repeated for every voxel?**

We are going to use the same contrasts introduced above (plotted below).

In [ ]:
# Plot the contrasts
for k, contrast_id in enumerate(['left - right click', 'audio - video']):
    plt.subplot(2,1,k+1)
    plt.bar(np.arange(len(contrasts[contrast_id])),
            contrasts[contrast_id])
    plt.plot([0, len(contrasts[contrast_id])-1], [0, 0], 'k')
    plt.xlim(-0.5, len(contrasts[contrast_id])-0.5)
    plt.title(contrast_id)
plt.tight_layout()

The following line of code uses a function from nistats, a Python package, to estimate the general linear model (GLM) on all voxels.

In [ ]:
# Estimate GLM on all voxels
labels, estimates = run_glm(fMRI_data, design_matrix.values)

The function run_glm does actually a bit more than the simple ordinary least square (OLS) solution used above with the formula. Check the help of the function (by running the next cell). It uses the option with ar1, which stands fro auto-regressive model.  
**Q3-2 Why do we use an auto-regressive model for the estimation?**  
Note that the OLS can also be used (but it is not the default option of run_glm).

In [ ]:
run_glm?

To illustrated the temporal autocorrelation of the signal, we can plot the autocorrelation function for an example voxel. 

In [ ]:
# compute and plot the autocorrelation function of an example voxel
TR_range = range(1, 20)
auto_correlation = [np.corrcoef(example_voxel[:-lag], example_voxel[lag:])[0, 1]
                    for lag in TR_range]
plt.plot(TR_range, auto_correlation, '-o')
plt.ylabel('correlation')
plt.xlabel('lag (# scan)')

The following cell computes the t-value of the contrast. This is similar to the computation done by hand above.

In [ ]:
# Estimate contrast
contrast_id = 'left - right click'
contrast = compute_contrast(labels, estimates,
                            contrasts[contrast_id],
                            contrast_type='t')
t_val = masker.inverse_transform(contrast.stat())

We are now going to display the thresholded T-map.

In [ ]:
# Plot glass brain
plotting.plot_glass_brain(t_val, threshold=3, plot_abs=False,
                          colorbar=True, title=contrast_id)

**Q3-3 What is a glass brain representation?**  
The following cell renders the results on slices.

In [ ]:
# Plot slices
plotting.plot_stat_map(t_val, display_mode='z',
                       threshold=3.0, title=contrast_id,
                       cut_coords=8)

**Q3-4: What is the orientation of the slices presented above?**  

The following cell renders the results on the surface of the right hemisphere.

In [ ]:
# get canonical surface anatomy
fsaverage = datasets.fetch_surf_fsaverage()

# project the results onto the mesh of cortical surface
texture = surface.vol_to_surf(t_val, fsaverage.pial_right)

# plot
plotting.plot_surf_stat_map(
        fsaverage.infl_right, texture, hemi='right',
        title=contrast_id, colorbar=True,
        threshold=3.0, bg_map=fsaverage.sulc_right)

**Q3-5: What do the blue-ish and red-ish blobs correspond to?**  
**Q3-6: Interpret the results.**  

The next cell now estimate and report the "audio-visual" contrast.

In [ ]:
# Estimate contrast
contrast_id = 'audio - video'
contrast_AV = compute_contrast(labels, estimates,
                            contrasts[contrast_id],
                            contrast_type='t')
t_val_AV = masker.inverse_transform(contrast_AV.stat())
plotting.plot_glass_brain(t_val_AV, threshold=3, plot_abs=False,
                          colorbar=True, title=contrast_id)

**Q3-7: Interpret the results.**  

## Statistical significance and correction for multiple comparisons
The figure above is displayed with a threshold of +/- 3.  
**Q3-8: Change the code to set a more liberal and a more conservative threshold**  
**Q3-9: Do you think that some voxels are actually false positives? Why?**  

We are going to use parametric statistics (i.e. the assumption that the t-values follow a known distribution) to assess significance levels as p-values.  
The following cell shows the map thresholded at p<0.001 (two-sided test, controlling for both positive and negative effects).  
Technical detail: the t-values are transformed into z-values because it is more convenient to do so with the nistats library.  
The z-value corresponding to p<0.001 (two-sided test) uncorrected is displayed.

In [ ]:
z_vals = contrast_AV.z_score()
z_map = masker.inverse_transform(z_vals)
z_thd = norm.isf(0.001/2)
print(z_thd)
plotting.plot_glass_brain(z_map, threshold=z_thd, plot_abs=False,
                          colorbar=True, title=contrast_id+' uncorr.')

The next cell computes a new threshold that controls for a given false discovery rate (FDR). This estimation is done with the Benjamini-Hochberg procedure. For details on this procedure, and more generally, an introduction to multiple comparisons correction, see this [review by T. Nichols and S. Hayasaka, 2003](https://journals.sagepub.com/doi/pdf/10.1191/0962280203sm341ra).   
The threshold (z-value) that corrects for the false discovery rate at p<0.001 (two-sided test) is reported and used to threshold the map.  
**Q3-10: Interpret the difference between corrected and uncorrected maps**  
Note that in practice, FDR is a weak correction. See the review for more conservative corrections. 

In [ ]:
z_thd = fdr_threshold(np.abs(z_vals), 0.001/2)
print(z_thd)
plotting.plot_glass_brain(z_map, threshold=z_thd, plot_abs=False,
                          colorbar=True, title=contrast_id+' corr.')